In [1]:
import cv2
import numpy as np
import os

In [2]:
# opencv comes with a lot pre trained models for detecting objects like face, smiles, license plates etc as well
# as a trainer which we can use to train for detection of our own objects

# these can be found in opencv/data/haarcascade

# here we ll load haarcascade frontal face detection pre trained model

In [3]:
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

eyeCascade = cv2.CascadeClassifier('haarcascade_eye.xml')

In [4]:
from keras.models import load_model

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
model = load_model(os.path.abspath('saved_model') + '/keras_face_expres_model_new150.h5')

/usr/local/lib/python3.5/dist-packages/keras/models.py:291: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [ ]:
capture = cv2.VideoCapture(0)

# choosing codec for saving into video file

#fourcc = cv2.VideoWriter_fourcc(*'XVID')

# choosing file for output writing

#out = cv2.VideoWriter('output_face.avi', fourcc, 20.0, (640, 480))
font = cv2.FONT_HERSHEY_SIMPLEX
count = 0
classDicti = {0:'Angry',1:'Disgust',2:'Fear',3:'Happy',4:'Sad',5:'Surprise',6:'Neutral'}

while True:
    
    predictionArray = []
    ret, img = capture.read()
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    faces = faceCascade.detectMultiScale(gray, 1.3, 5)

    
    # the above returns positions of the rectangle around face
    
    for (x, y, w, h) in faces:
        
        cv2.rectangle(img, (x+22,y+12), (x+w-22, y+h-12), (255, 0, 0), 2)
    
        # now eyes will only be detected inside face, it shouldnt b detected outside,
        # so we limit the roi to face itself
        roiGray = gray[y+12:y+h-12, x+22:x+w-22]
        roiColor = img[y+12:y+h-12, x+22:x+w-22]
        
        eyes = eyeCascade.detectMultiScale(roiGray)
        
        #for (ex, ey, ew, eh) in eyes:
        
            #cv2.rectangle(roiColor, (ex,ey), (ex + ew, ey + eh), (0, 255, 0), 2)
        
        # or we could choose to lock the face when eyes are detected by changing
        # the box color
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(img, (x+22,y+12), (x+w-22, y+h-12), (0, 255, 0), 2)
    
    #out.write(img)
    cv2.imshow('img', img)
    
    c = cv2.waitKey(1)
    
    if 's' == chr(c & 255):
        
        cropFace = roiGray
        cropFaceResize = cv2.resize(cropFace, (48, 48))
        inputFace = np.reshape(cropFaceResize, (48, 48, 1))
        predictionArray.append(inputFace)
        predictionArray = np.array(predictionArray)
        
        predict = model.predict_classes(predictionArray)
        #print(predict[0])
        #cv2.imwrite('Detected_face' + str(count) + '.jpg', cropFace)
        cv2.putText(img, classDicti[predict[0]], (100,100),font, 3, (200,255,255), cv2.LINE_AA )
        #cv2.imwrite('Detected_face' + str(count) + '.jpg', img)
        cv2.imshow('img2', img)
        count +=1
    if 'q' == chr(c & 255):
        break
cv2.destroyAllWindows()              
capture.release()